# Aim of This Notebook


In this notebook, we want to import the "messstelle.csv" file to our Postgres Database

 we need follwing packages: 

* Pandas 
* Sqlalchemy


In [1]:
#importing the packages

import pandas as pd
from sqlalchemy import create_engine
import sqlalchemy
import credential_temp as  creds 



# Data location

 First, we need to create the path of our data. 


In [2]:
data_in_dir = r"../data/"

gw_station_fname = r"opendata.gw_messstelle.csv"

gw_station_pfname = data_in_dir + gw_station_fname

## Read the file
Don't forget to add the "delimiter", normal csv file's delimiter is "," but in our case it's ";", so we need to assign it.

In [3]:
df = pd.read_csv(gw_station_pfname, delimiter=';', header='infer')
df.head()

,sl_nr,messstelle_id,name,e32,n32,gw_stockwerk,grundstueck,gemeinde_id,gwhorizont_id,gwhorizont,...,beobachtung_wasserstand,eigentuemer,betreiber,filterlaenge_cm,sumpfrohrlaenge_cm,ausbaudurchmesser_mm,historischer_ruhe_wsp,einbaulaenge_cm,oberkante_filter_cm,unterkante_filter_cm
0,67530,32505929,UWB-Ddorf 01285,343064,5678019,1.0,,05111000,NaN,NaN,...,-,Stadt Düsseldorf ...,Stadt Düsseldorf ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,51044,10446746,60GP012303,292077,5645349,NaN,privat,NL000882,5,Zwischenmittel,...,-,Prov. Limburg (NL) ...,Prov. Limburg (NL) ...,200.0,NaN,NaN,NaN,16893.0,-3333.0,-3533.0
2,51070,87005323,58BP024606,287141,5684635,NaN,privat,NL001640,6D,Neurather Sand,...,-,Prov. Limburg (NL) ...,Prov. Limburg (NL) ...,500.0,300.0,NaN,NaN,32667.0,-29083.0,-29583.0
3,68442,80000290,Krinsend 0079 neu,309490,5678060,1.0,,05166024,NaN,NaN,...,-,Land NRW ...,NaN,NaN,NaN,NaN,NaN,1517.0,4832.0,4832.0
4,68518,86583852,WG102GM93-3,316741,5680237,NaN,privat,05166032,NaN,NaN,...,-,NiederrheinWasser GmbH (ehem. Niederheinwerke)...,NiederrheinWasser GmbH (ehem. Niederheinwerke)...,200.0,NaN,80.0,NaN,5400.0,1413.0,1213.0


## data type

e32 and n32 should be int if we want to create geometry columns from them

In [4]:
df.dtypes

sl_nr                             int64
messstelle_id                     int64
name                             object
e32                              object
n32                              object
gw_stockwerk                    float64
grundstueck                      object
gemeinde_id                      object
gwhorizont_id                    object
gwhorizont                       object
gwleiter_id                      object
gwleiter                         object
einrichtungsgrund                object
gwk_lage_auf_id                  object
gwk_lage_id                      object
gwk_monitoring_auf_id            object
gwk_monitoring_id                object
messprogramm                     object
turnus_wasserstand               object
freigabe_wstd                    object
freigabe_chemie                  object
freigabe_lage                    object
wasserstandsmessstelle           object
guetemessstelle                  object
im_wrrl_messnetz_chemie          object


## change some data types
* e32 to Int
* n32 to Int
* drop NA value

First step, we must drop the NA value.
Second step is some rows the last two digit are xx which means that the exact location is not avaliable. 
Now we need to replace this xx with two digit. These two xx could be in the range of 00 to 99. 
In here we replace the xx with 50

In [5]:
# drop NA
df = df.dropna(subset=['e32'])
df = df.dropna(subset=['n32'])

In [6]:
df.e32

0        343064
1        292077
2        287141
3        309490
4        316741
          ...  
71115    468938
71116    3338xx
71117    331861
71118    3604xx
71119    3379xx
Name: e32, Length: 71119, dtype: object

In [7]:
# replace all xx with 50

df.e32 = df['e32'].replace({'xx':'50'}, regex=True)
df.n32 = df['n32'].replace({'xx':'50'}, regex=True)

In [8]:
# convert e32 and n32 to int
df['e32'] = df['e32'].astype(int)
df['n32'] = df['n32'].astype(int)

### Now the e32 and n32 are int

In [9]:
df.dtypes

sl_nr                             int64
messstelle_id                     int64
name                             object
e32                               int32
n32                               int32
gw_stockwerk                    float64
grundstueck                      object
gemeinde_id                      object
gwhorizont_id                    object
gwhorizont                       object
gwleiter_id                      object
gwleiter                         object
einrichtungsgrund                object
gwk_lage_auf_id                  object
gwk_lage_id                      object
gwk_monitoring_auf_id            object
gwk_monitoring_id                object
messprogramm                     object
turnus_wasserstand               object
freigabe_wstd                    object
freigabe_chemie                  object
freigabe_lage                    object
wasserstandsmessstelle           object
guetemessstelle                  object
im_wrrl_messnetz_chemie          object


In [10]:
filt = df['n32'] < 100
df[filt]

,sl_nr,messstelle_id,name,e32,n32,gw_stockwerk,grundstueck,gemeinde_id,gwhorizont_id,gwhorizont,...,beobachtung_wasserstand,eigentuemer,betreiber,filterlaenge_cm,sumpfrohrlaenge_cm,ausbaudurchmesser_mm,historischer_ruhe_wsp,einbaulaenge_cm,oberkante_filter_cm,unterkante_filter_cm
3895,30991,118820000,WW.GREVEN.DAMM-WAMI,50,50,1.0,,NaN,NaN,NaN,...,-,NaN,NaN,NaN,NaN,NaN,NaN,2594.0,1945.0,1945.0
4510,47647,47202002,Sammelleitung 21-30,50,50,NaN,,NaN,NaN,NaN,...,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8338,31022,118840009,WW.TELGTE-WAMI,50,50,1.0,,NaN,NaN,NaN,...,-,NaN,NaN,NaN,NaN,NaN,NaN,2179.0,2859.0,2859.0
10889,30882,118260005,WW.LETTE-WAMI,50,50,1.0,,NaN,NaN,NaN,...,-,NaN,NaN,NaN,NaN,NaN,NaN,9065.0,-1974.0,-1974.0
13335,47709,68013504,WW.HALTERN-BRR E/F,50,50,NaN,,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13414,14582,68012007,WW.HALTERNWEST-MI,50,50,1.0,,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13430,46769,47247101,RM-Moers Gerdt,50,50,NaN,,NaN,NaN,NaN,...,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17689,47708,68013401,WW.HALTERN-BRR C/D,50,50,NaN,,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19697,31047,118880007,WW.VOHREN-DACK-WAMI,50,50,1.0,,NaN,NaN,NaN,...,-,NaN,NaN,NaN,NaN,NaN,NaN,786.0,4713.0,4713.0
22201,47658,47299009,RM-Bucholtwelmen,50,50,NaN,,NaN,NaN,NaN,...,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df.drop(df[df['e32'] < 100].index, inplace = True)
df.drop(df[df['n32'] < 100].index, inplace = True)

### check externally
in order to check the all rows externally we can write it in csv file

In [12]:
df.to_csv("test.csv", index = False)

Other columns have no problem and the data is ready to import to Database.

## write in Database
Now we want to import our df to postgres database.

* First we have to connect to our Database URL
* then we can write into the our database. we need to assign in which Schema we want to add our data frame. in this case I have added my data frame to the schema "hygrisc" which has been created before in the PGadmin.

In [13]:
# connect to out credential file

postgresurl = creds.URL


In [14]:
# write your sql url below
engine = sqlalchemy.create_engine(postgresurl)


In [15]:
%time df.to_sql(con=engine, name="messstelle", schema="hygrisc", if_exists="replace", index=False)

CPU times: total: 9.39 s
Wall time: 12.6 s


90

### Create geometry columns

Now everything is fine and our data has been imported to Database successfully. but still someting important is missing.
We have e32 and new columns but in order to map these location correclty and exatly in QGIS, we need geometry information. 

in order to create geometry columns from n32 and e32 the follwing step is required. 



In [16]:
quary = """ ALTER TABLE hygrisc.messstelle ADD COLUMN geom geometry(Point, 25832);
 UPDATE hygrisc.messstelle SET geom = ST_SetSRID(ST_MakePoint(e32, n32), 25832);"""

In [17]:
engine.execute(quary)

ProgrammingError: (psycopg2.errors.UndefinedObject) type "geometry" does not exist
LINE 1:  ALTER TABLE hygrisc.messstelle ADD COLUMN geom geometry(Poi...
                                                        ^

[SQL:  ALTER TABLE hygrisc.messstelle ADD COLUMN geom geometry(Point, 25832);
 UPDATE hygrisc.messstelle SET geom = ST_SetSRID(ST_MakePoint(e32, n32), 25832);]
(Background on this error at: https://sqlalche.me/e/14/f405)

Now everything is fine and the data has been imported to our database with geometry columns sucessfully. You can see the data in PGadmin as well. 

## Finish!

That's great, You have imported the table successfully in Postgres Database.